<font size=5 font color='blue'> Part I: Asian option pricing with 1000, 10000, 100000, 1000000 and 10000000 simulations, using antithetic variables and moment matching for variance reduction.

In [3]:
import numpy as np

# Parameters
S0 = 110.0     # Initial stock price
K = 100.0      # Strike price
r = 0.01       # Risk-free rate
sigma = 0.3    # Volatility
T = 1.0        # Time to maturity (1 year)
m = 12         # Number of monitoring points (monthly)
dt = T / m

def compute_asian_price(n_sims):
    """
    Compute the Asian call option price and its standard error using Monte Carlo simulation.
    This function generates its own set of Z values (with moment matching and antithetic variates)
    for each call, so that each n_sims uses its own Z-set.
    
    Parameters:
      n_sims: Total number of simulation paths (must be even)
      
    Returns:
      price_estimate: Monte Carlo estimate of the option price.
      std_error: Standard error of the estimate.
    """
    
    n_sims_half = n_sims // 2

    Z = np.random.randn(n_sims_half, m)
    
    # Moment matching: force sample mean=0 and std=1
    Z = (Z - Z.mean()) / Z.std()
    
    # Create antithetic variates
    Z_antithetic = -Z
    
    Z_all = np.vstack((Z, Z_antithetic))
    
    # Compute log increments for each time step
    increments = (r - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * Z_all
    
    # Compute cumulative log-returns for each simulation path
    log_paths = np.cumsum(increments, axis=1)
    
    S_paths = S0 * np.exp(log_paths)
    
    S_mean = np.mean(S_paths, axis=1)

    payoffs = np.maximum(S_mean - K, 0.0)

    discount = np.exp(-r * T)
    discounted_payoffs = discount * payoffs

    price_estimate = np.mean(discounted_payoffs)
    payoff_std = np.std(discounted_payoffs)
    std_error = payoff_std / np.sqrt(n_sims)
    
    return price_estimate, std_error

# Build a convergence table using different simulation counts.
sim_list = [1_000, 10_000, 100_000, 1_000_000, 10_000_000]

print(f"{'n_sims':>10s} | {'Price':>10s} | {'Std Error':>10s} | {'Error (±2*SE)':>18s}")
print("-" * 54)
for sims in sim_list:
    price, se = compute_asian_price(sims)
    print(f"{sims:10d} | {price:10.4f} | {se:10.4f} | ±{2*se:16.4f}")

    n_sims |      Price |  Std Error |      Error (±2*SE)
------------------------------------------------------
      1000 |    14.4003 |     0.5516 | ±          1.1032
     10000 |    13.8935 |     0.1653 | ±          0.3307
    100000 |    13.9723 |     0.0526 | ±          0.1051
   1000000 |    13.9599 |     0.0166 | ±          0.0332
  10000000 |    13.9515 |     0.0052 | ±          0.0105


<font size=5 font color='blue'> Part II: Asian option pricing with 1000, 10000, 100000, 1000000 and 10000000 simulations, using antithetic variables, moment matching and control variates for variance reduction.

<font size=4 font color='blue'> First, calculate the price of the Asian option based on the geometric average of asset prices, which has an analytical solution.

In [5]:
t = np.arange(1, m+1) * (T / m)
T_bar = t.mean()

In [8]:
sum_ = 0.0
for i in range(1, m+1):
    sum_ += (2*i - 1) * t[m - i]
sigma_bar_sq = (sigma**2 / (m**2 * T_bar)) * sum_

In [9]:
dividend = 0.5*sigma**2-0.5*sigma_bar_sq

In [10]:
from scipy.stats import norm
d = (np.log(S0/K)+(r-dividend+0.5*sigma_bar_sq)*T_bar)/np.sqrt(sigma_bar_sq*T_bar)

In [11]:
price_geo = np.exp(-dividend*T_bar-r*(T-T_bar))*S0*norm.cdf(d)-np.exp(-r*T)*K*norm.cdf(d-np.sqrt(sigma_bar_sq*T_bar))
price_geo

13.40575400389794

<font size=4, font color='blue'> Then use the geometric Asian option price as a control variate in simulating arithmetic Asian option price.

In [13]:
def compute_asian_price_with_control(n_sims, S0, K, r, sigma, T, m, C_geom_exact):
    """
    Parameters:
      n_sims       : total number of simulation paths
      S0, K, r, sigma, T, m: standard option parameters
      C_geom_exact : known closed-form price of the geometric Asian call

    Returns:
      price_est : control-variate adjusted option price estimate
      std_error : standard error of the estimator (with control variate)
      corr      : sample correlation coefficient between arithmetic and geometric payoffs
      VRF       : variance reduction factor = Var(original)/Var(control estimator)
    """
    n_half = n_sims // 2
    discount = np.exp(-r * T)

    Z = np.random.randn(n_half, m)
    Z = (Z - Z.mean()) / Z.std()

    Z_anti = -Z
    Z_all = np.vstack((Z, Z_anti))  

    dt = T / m
    increments = (r - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * Z_all
    log_paths = np.cumsum(increments, axis=1)
    S_paths = S0 * np.exp(log_paths)  

    S_arith = np.mean(S_paths, axis=1)
    S_geom  = np.exp(np.mean(np.log(S_paths), axis=1))

    payoff_arith = np.maximum(S_arith - K, 0)
    payoff_geom  = np.maximum(S_geom  - K, 0)

    # --- Discounted payoffs ---
    disc_arith = discount * payoff_arith
    disc_geom  = discount * payoff_geom

    # --- Control variate method ---
    # b* = cov(X, Y) / var(X)
    # Y = disc_arith, X = disc_geom, E[X] = C_geom_exact
    # Y_cv = Y - b*( X - E[X] )
    X = disc_geom
    Y = disc_arith
    X_mean = X.mean()
    Y_mean = Y.mean()

    cov_XY = np.mean((X - X_mean) * (Y - Y_mean))  
    var_X  = np.mean((X - X_mean)**2)            

    b_est = cov_XY / var_X
    Y_cv = Y - b_est * (X - C_geom_exact)

    price_est = np.mean(Y_cv)
    std_error = np.std(Y_cv) / np.sqrt(n_sims)

    # --- Correlation & Variance Reduction Factor ---
    std_X = np.std(X)
    std_Y = np.std(Y)
    corr = cov_XY / (std_X * std_Y)
    var_original = np.var(Y)
    var_cv = np.var(Y_cv)
    VRF = 1/(1-corr**2)

    return price_est, std_error, corr, VRF

In [14]:
sim_list = [1_000, 10_000, 100_000, 1_000_000, 10_000_000]


print(f"{'n_sims':>10s} | {'Price_CV':>10s} | {'StdError':>10s} | {'±2*SE':>10s} | {'Corr':>8s} | {'VRF':>8s}")
print("-" * 66)

for sims in sim_list:
    price, se, corr, vrf = compute_asian_price_with_control(
        n_sims=sims,
        S0=S0, K=K, r=r, sigma=sigma, T=T, m=m,
        C_geom_exact=price_geo
    )
    print(f"{sims:10d} | {price:10.4f} | {se:10.4f} | ±{2*se:8.4f} | {corr:8.4f} | {vrf:8.4f}")

    n_sims |   Price_CV |   StdError |      ±2*SE |     Corr |      VRF
------------------------------------------------------------------
      1000 |    13.9007 |     0.0165 | ±  0.0329 |   0.9995 | 1002.3470
     10000 |    13.9466 |     0.0059 | ±  0.0117 |   0.9994 | 773.9619
    100000 |    13.9548 |     0.0019 | ±  0.0038 |   0.9994 | 778.4845
   1000000 |    13.9509 |     0.0006 | ±  0.0012 |   0.9994 | 784.1605
  10000000 |    13.9514 |     0.0002 | ±  0.0004 |   0.9994 | 782.0758
